# Fiddler Examples: Creating a Fiddler Model Using Model Constructor

The fastest method for onboarding a model to Fiddler is to provide a small sample of your model's test or training data to the [Model.from_data()](https://docs.fiddler.ai/python-client-3-x/api-methods-30#from_data) method. This process is demonstrated in Fiddler's [Simple ML Monitoring Quick Start](https://docs.fiddler.ai/quickstart-notebooks/quick-start). This method will inspect your data and automatically infer a [ModelSchema](https://docs.fiddler.ai/python-client-3-x/api-methods-30#modelschema) object containing:

* The set of column names
* Column data types (mapping them to Fiddler [DataType](https://docs.fiddler.ai/python-client-3-x/api-methods-30#datatype))
* Minium and maximum values for numerics, which are used for [Range Violation](https://docs.fiddler.ai/product-guide/monitoring-platform/data-integrity-platform) boundaries
* The unique possible values for strings inferred as Category types, which are used for [Range Violation](https://docs.fiddler.ai/product-guide/monitoring-platform/data-integrity-platform) boundaries

Occasionally, you must override the inferred [ModelSchema](https://docs.fiddler.ai/python-client-3-x/api-methods-30#modelschema), which must be done before the model is created if you use only the Fiddler Python client. Note that Model schema edits are supported in the Fiddler UI. Typical overrides include changing the inferred [DataType](https://docs.fiddler.ai/python-client-3-x/api-methods-30#datatype), min/max values, and possible Category values, as described in the [Customizing your Model Schema guide](https://docs.fiddler.ai/client-guide/customizing-your-model-schema).

### Creating ModelSchema Manually

In situations where human intervention in the model onboarding process is inconvenient—especially in automated model onboarding pipelines—it is advisable to create the [ModelSchema](https://docs.fiddler.ai/python-client-3-x/api-methods-30#modelschema) manually within your onboarding script. Instead of using the convenience method [Model.from_data()](https://docs.fiddler.ai/python-client-3-x/api-methods-30#from_data), you should use the [Model](https://docs.fiddler.ai/python-client-3-x/api-methods-30#model) object [constructor](https://docs.fiddler.ai/python-client-3-x/api-methods-30#constructor-3) and supply a preconfigured set of Columns. The following script demonstrates this process.

In [ ]:
%pip install -q fiddler-client

import time as time

import pandas as pd
import fiddler as fdl

print(f'Running Fiddler Python client version {fdl.__version__}')

In [19]:
URL = ''  # Make sure to include the full URL (including https:// e.g. 'https://your_company_name.fiddler.ai').
TOKEN = ''

Constants for this example notebook, change as needed to create your own versions

In [ ]:
PROJECT_NAME = 'quickstart_examples'  # If the project already exists, the notebook will create the model under the existing project.
MODEL_NAME = 'bank_churn_model_constructor3'

STATIC_BASELINE_NAME = 'baseline_dataset'

# Sample data hosted on GitHub
PATH_TO_SAMPLE_CSV = 'https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/data/v3/churn_data_sample.csv'
PATH_TO_EVENTS_CSV = 'https://raw.githubusercontent.com/fiddler-labs/fiddler-examples/main/quickstart/data/v3/churn_production_data.csv'

Now just run the following to connect to your Fiddler environment.

In [ ]:
fdl.init(url=URL, token=TOKEN)

Create a new or retrieve an existing Project with the supplied project name.

In [ ]:
project = fdl.Project.get_or_create(name=PROJECT_NAME)

## Manually Configure ModelSchema Columns

Here we create a list of [Column](https://docs.fiddler.ai/python-client-3-x/api-methods-30#column) objects which we will use for populating our ModelSchema's columns collection. Note that columns will have different required parameters depending on their DataType. For example, numeric columns require min/max values, but those are not allowed for Category [DataType](https://docs.fiddler.ai/python-client-3-x/api-methods-30#datatype) columns which require a list of categories instead.  

In [ ]:
columns = [
    fdl.Column(
        name='customer_id',
        data_type=fdl.DataType.STRING,
        min=None,
        max=None,
    ),
    fdl.Column(
        name='creditscore',
        data_type=fdl.DataType.INTEGER,
        min=350,
        max=850,
    ),
    fdl.Column(
        name='geography',
        data_type=fdl.DataType.CATEGORY,
        categories=[
            'California',
            'Florida',
            'Hawaii',
            'Massachusetts',
            'New York',
            'Texas',
        ],
        min=None,
        max=None,
    ),
    fdl.Column(
        name='gender',
        data_type=fdl.DataType.CATEGORY,
        categories=['Female', 'Male', 'Nonbinary'],
        min=None,
        max=None,
    ),
    fdl.Column(
        name='age',
        data_type=fdl.DataType.INTEGER,
        categories=None,
        min=18,
        max=92,
    ),
    fdl.Column(
        name='tenure',
        data_type=fdl.DataType.INTEGER,
        categories=None,
        min=0,
        max=10,
    ),
    fdl.Column(
        name='balance',
        data_type=fdl.DataType.FLOAT,
        categories=None,
        min=0.0,
        max=250898.09,
    ),
    fdl.Column(
        name='numofproducts',
        data_type=fdl.DataType.INTEGER,
        categories=None,
        min=1,
        max=4,
    ),
    fdl.Column(
        name='hascrcard',
        data_type=fdl.DataType.INTEGER,
        categories=None,
        min=0,
        max=1,
    ),
    fdl.Column(
        name='isactivemember',
        data_type=fdl.DataType.INTEGER,
        categories=None,
        min=0,
        max=1,
    ),
    fdl.Column(
        name='estimatedsalary',
        data_type=fdl.DataType.FLOAT,
        categories=None,
        min=11.58,
        max=199992.48,
    ),
    fdl.Column(
        name='predicted_churn',
        data_type=fdl.DataType.FLOAT,
        categories=None,
        min=8.339427867148919e-05,
        max=0.9999984102948216,
    ),
    fdl.Column(
        name='churn',
        data_type=fdl.DataType.CATEGORY,
        categories=['no', 'yes'],
        min=None,
        max=None,
    ),
    fdl.Column(
        name='timestamp',
        data_type=fdl.DataType.INTEGER,
        categories=None,
        min=1710428231855,
        max=1711033031855,
    ),
]

## Using the Model Constructor

Now that we have our columns configured, we can create an instance of [ModelSchema](https://docs.fiddler.ai/python-client-3-x/api-methods-30#modelschema) and then assign the rest of the Model parameters. 

In [ ]:
schema = fdl.ModelSchema(columns=columns)

input_columns = [
    'creditscore',
    'geography',
    'gender',
    'age',
    'tenure',
    'balance',
    'numofproducts',
    'hascrcard',
    'isactivemember',
    'estimatedsalary',
]
output_column = 'predicted_churn'
target_column = 'churn'
metadata_columns = ['customer_id', 'timestamp']

model_spec = fdl.ModelSpec(
    inputs=input_columns,
    outputs=[output_column],
    targets=[
        target_column
    ],  # Note: only a single Target column is allowed, use metadata columns and custom metrics for additional targets
    metadata=metadata_columns,
)

model = fdl.Model(
    name=MODEL_NAME,
    project_id=project.id,
    task=fdl.ModelTask.BINARY_CLASSIFICATION,
    task_params=fdl.ModelTaskParams(target_class_order=['no', 'yes']),
    schema=schema,
    spec=model_spec,
    event_id_col='customer_id',
    event_ts_col='timestamp',
)

### Check the ModelSchema

In [ ]:
# Check schema
model.schema

### Create the Model

In [ ]:
model.create()

## Next Steps

From this point, you may proceed as usual with model onboarding by (optionally) publishing a preproduction [Baseline](https://docs.fiddler.ai/client-guide/creating-a-baseline-dataset) dataset, creating [Alerts](https://docs.fiddler.ai/client-guide/alerts-with-fiddler-client), and [publishing](https://docs.fiddler.ai/client-guide/publishing-production-data) production events.

**Questions?**  
  
Check out [our docs](https://docs.fiddler.ai/) for a more detailed explanation of what Fiddler has to offer.